# Context Managers

Context managers are normally used to allocate and release resources automatically without needing to worry about about doing it yourself. You normally use the `with` keyword when dealing with CMs.

The most common example of a CM is when working with file IO. You normally see something like:
```python
with open('somefile.txt') as f:
    do something...
```
We show an example below.

In [2]:
with open('static/Genesis.txt') as f:
    print(next(f))

[1:1] In the beginning when God created the heavens and the earth,



In [1]:
from time import time

class Timer:
    def __enter__(self):
        self.start = time()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = time()
        print(self.end - self.start, 'seconds to run')

In [33]:
with Timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3250179290771484 seconds to run


In [34]:
from contextlib import contextmanager

@contextmanager
def timer():
    start = time()
    yield
    end = time()
    print(end - start, 'seconds to run')

In [35]:
with timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3074147701263428 seconds to run
